<a href="https://colab.research.google.com/github/data602sps/assignments/blob/master/05_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 7**

# **Weeks 8 & 9 - Pandas**
* In this homework assignment, you will explore and analyze a public dataset of your choosing. Since this assignment is “open-ended” in nature, you are free to expand upon the requirements below. However, you must meet the minimum requirments as indicated in each section. 

* You must use Pandas as the **primary tool** to process your data.

* The preferred method for this analysis is in a .ipynb file. Feel free to use whichever platform of your choosing.  
 * https://www.youtube.com/watch?v=inN8seMm7UI (Getting started with Colab).

* Your data should need some "work", or be considered "dirty".  You must show your skills in data cleaning/wrangling.

### **Some data examples:**
•	https://www.data.gov/

•	https://opendata.cityofnewyork.us/

•	https://datasetsearch.research.google.com/

•	https://archive.ics.uci.edu/ml/index.php

### **Resources:**

•	https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html 

•	https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html


### **Headings or comments**
**You are required to make use of comments, or headings for each section.  You must explain what your code is doing, and the results of running your code.**  Act as if you were giving this assignment to your manager - you must include clear and descriptive information for each section.

### **You may work as a group or individually on this assignment.**


# Introduction

In this section, please describe the dataset you are using.  Include a link to the source of this data.  You should also provide some explanation on why you choose this dataset.

The dataset I am using is a collection of 1,000 IMDb movie reviews. This dataframe contains information and descriptions for each movie, as well as their IMDb ratings. The data was taken from a [Kaggle dataset](https://www.kaggle.com/datasets/akashkotal/imbd-top-1000-with-description). 

I also supplemented this data with [another Kaggle dataset](https://www.kaggle.com/datasets/dk123891/10000-movies-data?select=data.csv) of IMDb movie ratings, as I wanted to add columns for genre and viewer rating for each movie. While this dataset technically has all the same columns as the first dataset, it is much cleaner and therefore not as much an exercise in data wrangling. For this reason, I will only use the `Movie Name`, `Year of Release`, `Genre`, and `Certification` columns on this dataset. 

**Data Dictionary:**

**movies_df**:
Column|Description
--|--
Movie Name|The title of the movie
Year of Release|The year the movie came out
Watch Time|The runtime in minutes of the movie
Meatscore of movie|The metascore of the movie (incorrecly labeled here as meatscore)
Votes|The number of votes contributing to the movie rating
Gross|Box office revenue
Description|iMDB description of the movie 

**genres_df:**
Column|Description
--|--
Movie Name|The title of the movie
Year of Release|The year the movie came out
Genre|List of genre categories the movie falls into
Certification|Audience age rating for the movie


______________
# Data Exploration
Import your dataset into your .ipynb, create dataframes, and explore your data.  

Include: 

* Summary statistics means, medians, quartiles, 
* Missing value information
* Any other relevant information about the dataset.  



In [141]:
import pandas as pd
import re
from datetime import datetime
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# load data
movies_df = pd.read_csv('https://raw.githubusercontent.com/ShanaFarber/cuny-sps/master/DATA_602/assignment7/top-100-imdb-movies.csv', index_col=0)
movies_df2 = pd.read_csv('https://raw.githubusercontent.com/ShanaFarber/cuny-sps/master/DATA_602/assignment7/sup-imdb-movies.csv', index_col=0)
genres_df = movies_df2[['Movie Name', 'Year of Release', 'Genre', 'Certification']]  # only use specific columns

movies_df.head()    # preview movie data

,Movie Name,Year of Release,Watch Time,Movie Rating,Meatscore of movie,Votes,Gross,Description
0,The Shawshank Redemption,(1994),142 min,9.3,81,"34,709",$28.34M,Two imprisoned men bond over a number of years...
1,The Godfather,(1972),175 min,9.2,100,"34,709",$134.97M,The aging patriarch of an organized crime dyna...
2,The Dark Knight,(2008),152 min,9.0,84,"34,709",$534.86M,When the menace known as the Joker wreaks havo...
3,The Lord of the Rings: The Return of the King,(2003),201 min,9.0,94,"34,709",$377.85M,Gandalf and Aragorn lead the World of Men agai...
4,Schindler's List,(1993),195 min,9.0,94,"34,709",$96.90M,"In German-occupied Poland during World War II,..."


In [142]:
genres_df.head()

,Movie Name,Year of Release,Genre,Certification
0,The Shawshank Redemption,1994,['Drama'],R
1,The Godfather,1972,"['Crime', ' Drama']",R
2,Ramayana: The Legend of Prince Rama,1993,"['Animation', ' Action', ' Adventure']",PG
3,The Chaos Class,1975,"['Comedy', ' Drama']",NaN
4,The Dark Knight,2008,"['Action', ' Crime', ' Drama']",PG-13


Let's check to see how many null values are in each column of these datasets. 

In [143]:
movies_df.info()    # check if null values

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Movie Name          1000 non-null   object 
 1   Year of Release     1000 non-null   object 
 2   Watch Time          1000 non-null   object 
 3   Movie Rating        1000 non-null   float64
 4   Meatscore of movie  1000 non-null   object 
 5   Votes               1000 non-null   object 
 6   Gross               1000 non-null   object 
 7   Description         1000 non-null   object 
dtypes: float64(1), object(7)
memory usage: 70.3+ KB


There are 1,00 entries in the movies dataset. Every column has a not-null count of 1000 so there are no missing values in this dataset. 

Currently, all of the datatypes are objects except for `Movie Rating` which is a float. We can get the descriptive statistics for this column. 

In [144]:
movies_df.describe()

,Movie Rating
count,1000.000000
mean,7.962700
std,0.276759
min,7.600000
25%,7.700000
50%,7.900000
75%,8.100000
max,9.300000


The mean rating is 7.96. The median is 7.9. The interquartile range is 7.7-8.1, so 50% of ratings fall within this range. 

Let's take a look at some of the non-numeric columns. 

In [145]:
movies_df[['Movie Name', 'Year of Release', 'Watch Time', 'Meatscore of movie', 'Votes', 'Gross']].describe()

,Movie Name,Year of Release,Watch Time,Meatscore of movie,Votes,Gross
count,1000,1000,1000,1000,1000,1000
unique,996,121,141,62,1,746
top,The Girl with the Dragon Tattoo,(2004),130 min,****,"34,709",%^%^%^
freq,2,29,24,164,1000,153


Here we can see that not all the movie titles are unique. This can be because the same movie may have been released in different years, for example if there was a remake of the movie in a later year. We also see that `Gross` has a highest value of "%^%^%^" which is probably due to some error in scraping. Also, the top metascore is "****" probably due to the same reasons. To convert these columns to numeric, we will need to remove these values. 

It also seems that the `votes` column consists only of 34,709. This must have been an error in scraping. 

In [146]:
genres_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Movie Name       10000 non-null  object
 1   Year of Release  10000 non-null  int64 
 2   Genre            10000 non-null  object
 3   Certification    9631 non-null   object
dtypes: int64(1), object(3)
memory usage: 390.6+ KB


There are 10,000 entries in the genres dataframe. There are null values in the certification column, so there are some movies which we do not know the age rating for. 

We can also see that `Year of Release` is coded as an integer here. In order to merge the dataframes, we will need to have the datatype for this column match in both **movies_df** and **genres_df**. 

In [147]:
genres_df[['Movie Name', 'Genre', 'Certification']].describe()

,Movie Name,Genre,Certification
count,10000,10000,9631
unique,9632,425,24
top,The Mummy,"['Comedy', ' Drama', ' Romance']",R
freq,4,481,4048


# Data Wrangling
Create a subset of your original data and perform the following.  

1. Modify multiple column names.

2. Look at the structure of your data – are any variables improperly coded? Such as strings or characters? Convert to correct structure if needed.

3. Fix missing and invalid values in data.

4. Create new columns based on existing columns or calculations.

5. Drop column(s) from your dataset.

6. Drop row(s) from your dataset.

7. Sort your data based on multiple variables. 

8. Filter your data based on some condition. 

9. Convert all the string values to upper or lower cases in one column.

10. Check whether numeric values are present in a given column of your dataframe.

11. Group your dataset by one column, and get the mean, min, and max values by group. 
  * groupby()
  * agg() or .apply()

12. Group your dataset by two columns and then sort the aggregated results within the groups. 

**You are free (and should) to add on to these questions.  Please clearly indicate in your assignment your answers to these questions.**

Let's start with the `movies_df` dataframe.

First, let's start with fixing up the names of the columns. 

In [148]:
# fix spelling of metascore column
cleaned_movies_df = movies_df.rename({'Meatscore of movie': 'Metascore'}, axis=1)

# fix column names
cleaned_movies_df.columns = cleaned_movies_df.columns.str.replace(' ', '_').str.lower()     # make snake case
cleaned_movies_df = cleaned_movies_df.rename({'year_of_release':'release_year', 'watch_time':'runtime_mins'}, axis=1)      # change some names

cleaned_movies_df.head()

,movie_name,release_year,runtime_mins,movie_rating,metascore,votes,gross,description
0,The Shawshank Redemption,(1994),142 min,9.3,81,"34,709",$28.34M,Two imprisoned men bond over a number of years...
1,The Godfather,(1972),175 min,9.2,100,"34,709",$134.97M,The aging patriarch of an organized crime dyna...
2,The Dark Knight,(2008),152 min,9.0,84,"34,709",$534.86M,When the menace known as the Joker wreaks havo...
3,The Lord of the Rings: The Return of the King,(2003),201 min,9.0,94,"34,709",$377.85M,Gandalf and Aragorn lead the World of Men agai...
4,Schindler's List,(1993),195 min,9.0,94,"34,709",$96.90M,"In German-occupied Poland during World War II,..."


Now let's clean up the column datatypes. 

In [149]:
# function to clean numeric columns
def numeric_clean(input_string):
    try:
        if '.' not in input_string:
            return int(re.sub('[A-Za-z(),]', '', input_string))    # convert columns to integer
        else:
            return float(re.sub('[$^%M]', '', input_string))    # convert decimal columns to float
    except ValueError:
        return np.nan   # fill errors with NA

# clean year column
cleaned_movies_df.release_year = cleaned_movies_df.release_year.apply(numeric_clean)
cleaned_movies_df.release_year = cleaned_movies_df.release_year.apply(lambda x: datetime(year=x, month=1, day=1).year)

# clean runtime column
cleaned_movies_df.runtime_mins = cleaned_movies_df.runtime_mins.apply(numeric_clean)

# clean metascores
cleaned_movies_df.metascore = cleaned_movies_df.metascore.apply(numeric_clean)

# clean gross
cleaned_movies_df.gross = cleaned_movies_df.gross.apply(lambda x: numeric_clean(x)*1000000) # clean and multiply by a mill

cleaned_movies_df.head()

,movie_name,release_year,runtime_mins,movie_rating,metascore,votes,gross,description
0,The Shawshank Redemption,1994,142,9.3,81.0,"34,709",28340000.0,Two imprisoned men bond over a number of years...
1,The Godfather,1972,175,9.2,100.0,"34,709",134970000.0,The aging patriarch of an organized crime dyna...
2,The Dark Knight,2008,152,9.0,84.0,"34,709",534860000.0,When the menace known as the Joker wreaks havo...
3,The Lord of the Rings: The Return of the King,2003,201,9.0,94.0,"34,709",377850000.0,Gandalf and Aragorn lead the World of Men agai...
4,Schindler's List,1993,195,9.0,94.0,"34,709",96900000.0,"In German-occupied Poland during World War II,..."


As mentioned above, the `votes` column consists only of 34,709, which must have been an error from scraping the data. Let's remove this column. 

In [150]:
# drop votes
cleaned_movies_df = cleaned_movies_df.drop('votes', axis=1)
cleaned_movies_df.head()

,movie_name,release_year,runtime_mins,movie_rating,metascore,gross,description
0,The Shawshank Redemption,1994,142,9.3,81.0,28340000.0,Two imprisoned men bond over a number of years...
1,The Godfather,1972,175,9.2,100.0,134970000.0,The aging patriarch of an organized crime dyna...
2,The Dark Knight,2008,152,9.0,84.0,534860000.0,When the menace known as the Joker wreaks havo...
3,The Lord of the Rings: The Return of the King,2003,201,9.0,94.0,377850000.0,Gandalf and Aragorn lead the World of Men agai...
4,Schindler's List,1993,195,9.0,94.0,96900000.0,"In German-occupied Poland during World War II,..."


Now let's clean up `genres_df`.

In [151]:
cleaned_genres_df = genres_df
cleaned_genres_df['Genre'] = cleaned_genres_df['Genre'].apply(lambda x: x.strip('(\[|\]|\s*)').replace(' ', '').upper().split(','))   # turn genres column into actual list and change case
cleaned_genres_df.columns = cleaned_genres_df.columns.str.replace(' ', '_').str.lower()     # make columns snake case
cleaned_genres_df = cleaned_genres_df.rename({'year_of_release':'release_year'}, axis=1)

cleaned_genres_df.head()

,movie_name,release_year,genre,certification
0,The Shawshank Redemption,1994,['DRAMA'],R
1,The Godfather,1972,"['CRIME', 'DRAMA']",R
2,Ramayana: The Legend of Prince Rama,1993,"['ANIMATION', 'ACTION', 'ADVENTURE']",PG
3,The Chaos Class,1975,"['COMEDY', 'DRAMA']",NaN
4,The Dark Knight,2008,"['ACTION', 'CRIME', 'DRAMA']",PG-13


Now let's merge the dataframes. 

In [152]:
merged_df = pd.merge(cleaned_movies_df, cleaned_genres_df)

merged_df.head()

,movie_name,release_year,runtime_mins,movie_rating,metascore,gross,description,genre,certification
0,The Shawshank Redemption,1994,142,9.3,81.0,28340000.0,Two imprisoned men bond over a number of years...,['DRAMA'],R
1,The Godfather,1972,175,9.2,100.0,134970000.0,The aging patriarch of an organized crime dyna...,"['CRIME', 'DRAMA']",R
2,The Dark Knight,2008,152,9.0,84.0,534860000.0,When the menace known as the Joker wreaks havo...,"['ACTION', 'CRIME', 'DRAMA']",PG-13
3,The Lord of the Rings: The Return of the King,2003,201,9.0,94.0,377850000.0,Gandalf and Aragorn lead the World of Men agai...,"['ACTION', 'ADVENTURE', 'DRAMA']",PG-13
4,Schindler's List,1993,195,9.0,94.0,96900000.0,"In German-occupied Poland during World War II,...","['BIOGRAPHY', 'DRAMA', 'HISTORY']",R


Let's also create another column for the runtime in hours based on `runtime_mins`.

In [153]:
merged_df['runtime_hrs'] = merged_df.runtime_mins.apply(lambda x: round(x/60, 2))

merged_df.head()

,movie_name,release_year,runtime_mins,movie_rating,metascore,gross,description,genre,certification,runtime_hrs
0,The Shawshank Redemption,1994,142,9.3,81.0,28340000.0,Two imprisoned men bond over a number of years...,['DRAMA'],R,2.37
1,The Godfather,1972,175,9.2,100.0,134970000.0,The aging patriarch of an organized crime dyna...,"['CRIME', 'DRAMA']",R,2.92
2,The Dark Knight,2008,152,9.0,84.0,534860000.0,When the menace known as the Joker wreaks havo...,"['ACTION', 'CRIME', 'DRAMA']",PG-13,2.53
3,The Lord of the Rings: The Return of the King,2003,201,9.0,94.0,377850000.0,Gandalf and Aragorn lead the World of Men agai...,"['ACTION', 'ADVENTURE', 'DRAMA']",PG-13,3.35
4,Schindler's List,1993,195,9.0,94.0,96900000.0,"In German-occupied Poland during World War II,...","['BIOGRAPHY', 'DRAMA', 'HISTORY']",R,3.25


Let's take a look at the structure of the new dataframe. 

In [154]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_name     978 non-null    object 
 1   release_year   978 non-null    int64  
 2   runtime_mins   978 non-null    int64  
 3   movie_rating   978 non-null    float64
 4   metascore      819 non-null    float64
 5   gross          797 non-null    float64
 6   description    978 non-null    object 
 7   genre          978 non-null    object 
 8   certification  968 non-null    object 
 9   runtime_hrs    978 non-null    float64
dtypes: float64(4), int64(2), object(4)
memory usage: 76.5+ KB


There are 978 entries in this dataframe and 9 columns. 

`metascore` and `gross` have null values. 

What are the top five grossing movies?

In [155]:
# sort highest - get top 5
top_5_gross = merged_df.sort_values('gross', ascending=False).head()
top_5_gross

,movie_name,release_year,runtime_mins,movie_rating,metascore,gross,description,genre,certification,runtime_hrs
598,Star Wars: Episode VII - The Force Awakens,2015,138,7.8,80.0,936660000.0,"As a new threat to the galaxy rises, Rey, a de...","['ACTION', 'ADVENTURE', 'SCI-FI']",PG-13,2.30
63,Avengers: Endgame,2019,181,8.4,78.0,858370000.0,After the devastating events of Avengers: Infi...,"['ACTION', 'ADVENTURE', 'DRAMA']",PG-13,3.02
97,Spider-Man: No Way Home,2021,148,8.3,71.0,804750000.0,"With Spider-Man's identity now revealed, Peter...","['ACTION', 'ADVENTURE', 'FANTASY']",PG-13,2.47
632,Avatar,2009,162,7.8,83.0,760510000.0,A paraplegic Marine dispatched to the moon Pan...,"['ACTION', 'ADVENTURE', 'FANTASY']",PG-13,2.70
64,Avengers: Infinity War,2018,149,8.4,68.0,678820000.0,The Avengers and their allies must be willing ...,"['ACTION', 'ADVENTURE', 'SCI-FI']",PG-13,2.48


What are the top five movies based on rating?

In [156]:
top_5_rating = merged_df.head()
top_5_rating

,movie_name,release_year,runtime_mins,movie_rating,metascore,gross,description,genre,certification,runtime_hrs
0,The Shawshank Redemption,1994,142,9.3,81.0,28340000.0,Two imprisoned men bond over a number of years...,['DRAMA'],R,2.37
1,The Godfather,1972,175,9.2,100.0,134970000.0,The aging patriarch of an organized crime dyna...,"['CRIME', 'DRAMA']",R,2.92
2,The Dark Knight,2008,152,9.0,84.0,534860000.0,When the menace known as the Joker wreaks havo...,"['ACTION', 'CRIME', 'DRAMA']",PG-13,2.53
3,The Lord of the Rings: The Return of the King,2003,201,9.0,94.0,377850000.0,Gandalf and Aragorn lead the World of Men agai...,"['ACTION', 'ADVENTURE', 'DRAMA']",PG-13,3.35
4,Schindler's List,1993,195,9.0,94.0,96900000.0,"In German-occupied Poland during World War II,...","['BIOGRAPHY', 'DRAMA', 'HISTORY']",R,3.25


What are the bottom five movies based on rating?

In [157]:
bottom_5_rating = merged_df.tail()
bottom_5_rating

,movie_name,release_year,runtime_mins,movie_rating,metascore,gross,description,genre,certification,runtime_hrs
973,Sabrina,1954,113,7.6,72.0,NaN,A playboy becomes interested in the daughter o...,"['COMEDY', 'DRAMA', 'ROMANCE']",Passed,1.88
974,From Here to Eternity,1953,118,7.6,85.0,30500000.0,"At a U.S. Army base in 1941 Hawaii, a private ...","['DRAMA', 'ROMANCE', 'WAR']",Passed,1.97
975,Snow White and the Seven Dwarfs,1937,83,7.6,95.0,184930000.0,Exiled into the dangerous forest by her wicked...,"['ANIMATION', 'ADVENTURE', 'FAMILY']",Approved,1.38
976,The 39 Steps,1935,86,7.6,93.0,NaN,A man in London tries to help a counter-espion...,"['CRIME', 'MYSTERY', 'THRILLER']",Approved,1.43
977,The Invisible Man,1933,71,7.6,87.0,NaN,"A scientist finds a way of becoming invisible,...","['HORROR', 'SCI-FI']",TV-PG,1.18


Of those movies with a metascore of 100, which were the highest grossing?

In [158]:
metascore_100 = merged_df[merged_df.metascore == 100]      # filter for metascore of 100
highest_metascore_100 = metascore_100.sort_values('gross', ascending=False)     # sort descending gross
highest_metascore_100 = highest_metascore_100.dropna()      # drop rows missing gross
highest_metascore_100

,movie_name,release_year,runtime_mins,movie_rating,metascore,gross,description,genre,certification,runtime_hrs
1,The Godfather,1972,175,9.2,100.0,134970000.0,The aging patriarch of an organized crime dyna...,"['CRIME', 'DRAMA']",R,2.92
118,Lawrence of Arabia,1962,218,8.3,100.0,44820000.0,"The story of T.E. Lawrence, the English office...","['ADVENTURE', 'BIOGRAPHY', 'DRAMA']",Approved,3.63
54,Rear Window,1954,112,8.5,100.0,36760000.0,A wheelchair-bound photographer spies on his n...,"['MYSTERY', 'THRILLER']",PG,1.87
484,Boyhood,2014,165,7.9,100.0,25380000.0,"The life of Mason, from early childhood to his...",['DRAMA'],R,2.75
564,Notorious,1946,102,7.9,100.0,10460000.0,The daughter of a convicted Nazi spy is asked ...,"['DRAMA', 'FILM-NOIR', 'ROMANCE']",Not Rated,1.70
278,Fanny and Alexander,1982,188,8.1,100.0,4970000.0,Two young Swedish children in the 1900s experi...,['DRAMA'],R,3.13
265,Three Colors: Red,1994,99,8.1,100.0,4040000.0,A model discovers a retired judge is keen on i...,"['DRAMA', 'MYSTERY', 'ROMANCE']",R,1.65
122,Vertigo,1958,128,8.3,100.0,3200000.0,A former San Francisco police detective juggle...,"['MYSTERY', 'ROMANCE', 'THRILLER']",PG,2.13
128,Citizen Kane,1941,119,8.3,100.0,1590000.0,Following the death of publishing tycoon Charl...,"['DRAMA', 'MYSTERY']",PG,1.98
55,Casablanca,1942,102,8.5,100.0,1020000.0,A cynical expatriate American cafe owner strug...,"['DRAMA', 'ROMANCE', 'WAR']",PG,1.70


The Godfather (1972) is the highest grossing movie of those with a metascore of 100. 

What is the longest film? The shortest film?

In [159]:
# longest
longest = merged_df.sort_values('runtime_mins', ascending=False).head(1)
longest

,movie_name,release_year,runtime_mins,movie_rating,metascore,gross,description,genre,certification,runtime_hrs
145,Gangs of Wasseypur,2012,321,8.2,89.0,NaN,A clash between Sultan and Shahid Khan leads t...,"['ACTION', 'COMEDY', 'CRIME']",Not Rated,5.35


In [160]:
# shortest
shortest = merged_df.sort_values('runtime_mins').head(1)
shortest

,movie_name,release_year,runtime_mins,movie_rating,metascore,gross,description,genre,certification,runtime_hrs
200,Sherlock Jr.,1924,45,8.2,NaN,980000.0,"A film projectionist longs to be a detective, ...","['ACTION', 'COMEDY', 'ROMANCE']",Passed,0.75


The longest movie in the dataset is Gangs of Wasseypur (2012) with a runtime of 321 minutes (about 5.35 hours). The shortest movie in the dataset is Sherlock Jr. with a runtime of 45 minutes (about 0.75 hours). 

How many movies of each audience rating (certification) are there?

In [161]:
cert_counts = pd.DataFrame(merged_df['certification'].value_counts()).reset_index()
cert_counts

,certification,count
0,R,353
1,PG-13,166
2,Not Rated,165
3,PG,141
4,Passed,47
5,G,39
6,Approved,32
7,TV-MA,7
8,TV-PG,4
9,TV-14,3


Majority of movies in the dataframe are rated R. 

What is the average rating for movies rated R?

In [162]:
r_rated = merged_df[merged_df.certification == 'R']
round(r_rated.movie_rating.agg('mean'),1)

7.9

What is the average rating for movies for each certification?. What is the min rating? Max rating?

In [163]:
merged_df.groupby('certification').agg({'movie_rating':['mean','min','max']})

movie_rating          
                      mean  min  max
certification                       
16+               8.000000  7.8  8.2
18+               8.300000  8.3  8.3
Approved          8.062500  7.6  9.0
G                 8.002564  7.6  8.5
GP                8.000000  7.9  8.1
M                 7.600000  7.6  7.6
M/PG              8.200000  8.2  8.2
NC-17             7.700000  7.7  7.7
Not Rated         8.015152  7.6  8.6
PG                7.943972  7.6  8.7
PG-13             7.917470  7.6  9.0
Passed            7.982979  7.6  8.4
R                 7.939093  7.6  9.3
TV-14             8.133333  7.8  8.4
TV-MA             8.242857  7.7  8.9
TV-PG             7.775000  7.6  7.9
Unrated           7.800000  7.8  7.8

What is the average rating of movies per certification for each year?

In [164]:
# find rating averages per cert per year
year_cert_rating = merged_df.groupby(['release_year','certification']).agg({'movie_rating':'mean'})
year_cert_rating

movie_rating
release_year certification              
1920         Not Rated               8.0
1921         Passed                  8.3
1922         Not Rated               7.9
1924         Passed                  8.2
1925         Not Rated               8.0
...                                  ...
2021         TV-14                   8.4
             TV-MA                   8.7
2022         Not Rated               8.2
             PG-13                   8.1
             R                       8.2

[336 rows x 1 columns]

What is the highest rated certification and year?

In [165]:
# sort by rating
year_cert_rating.sort_values('movie_rating', ascending=False)

,,movie_rating
release_year,certification,
1966,Approved,8.8
2020,TV-MA,8.7
2021,TV-MA,8.7
1946,PG,8.6
1960,R,8.5
...,...,...
1954,Passed,7.6
1988,PG-13,7.6
1968,Passed,7.6


# Conclusions  

After exploring your dataset, provide a short summary of what you noticed from this dataset.  What would you explore further with more time?

#### Exploration of Initial Structure

When exploring the initial structure of the dataframe, it seemed that there were no null values present in the dataset. However, upon further exploration, there were a number of entries which seemed to have been improperly pulled from the IMDb website. Therefore, there were a number of invalid data entries. In order to be able to convert the desired columns to integer format, these values needed to be remove and converted to null values. There was also an entire column which had to be discarded due to the fact that the scraper pulled the same value for the entire column. 

Also during the initial exploration, I noticed that the movie names were not all unique. There were some instances of movies with the same title released in different years (probably remakes), so the genres dataframe that I chose to merge on needed to have the year column as well so that I would be able to merge on `movie_name` and `release_year`.

I also realized that the `genre` column, even though each entry was in the form of a list, did not read into python in list format. Therefore, I had to clean up this column and replace the entries with their list counterparts. 

#### Exploration of the Data

- The five highest grossing movies were _Star Wars: Episode VII - The Force Awakens_, _Avengers: Endgame_, _Spider-Man: No Way Home_, _Avatar_, and _Avengers: Infinity War_.
- The five highest rated movies were _Shawshank Redemption_, _The Godfather_, _The Dark Knight_, _The Lord of the Rings: The Return of the King_, and _Schinler's List_. 
- The five lowest rated movies were _Sabrina_, _From Here to Eternity_, _Snow White and the Seven Dwarfs_, _The 39 Steps_, and _The Invisible Man_, all with scores of 7.6. 
- The highest grossing movie of those with a metascore of 100 was _The Godfather_, which grossed $134.97M.
- The longest film in the dataset is _Gangs of Wasseypur_.
- The shortest film in the dataset is _Sherlock Jr._
- Most of the movies in the dataset are rated R (353 movies). The second highest certification is PG-13 (166 movies). 
- The average rating of rated R movies in the dataset is 7.94.
- The average rating of rated PG-13 movies in the dataset is 7.92. 

#### Further Exploration

To further explore the data, I would explode the `genre` column and perform aggregate functions group by `genre`, `release_year`, and `certification`. I would also create time series plots to track `movie_rating`, `runtime_mins`, and `gross` over the years.

I would also like to try to create a movie recommender system based on movie ratings. For this, I would want to come up with some sort of weighted rating based also on the number of votes that a movie was given (i.e. the number of people who rated). For this, I would need to 
